Deteksi dan filtering(rasio green dan chm)

In [17]:
from ultralytics import YOLO
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point
import os
import rasterio
import numpy as np
from sklearn.cluster import DBSCAN
import types

Image.MAX_IMAGE_PIXELS = None  # Menghilangkan batasan piksel

# Fungsi untuk memotong gambar menjadi beberapa window tanpa padding
def sliding_window(image, step_size, window_size):
    # Loop melalui koordinat y dengan langkah sebesar step_size
    for y in range(0, image.height, step_size):
        # Loop melalui koordinat x dengan langkah sebesar step_size
        for x in range(0, image.width, step_size):
            # Menghasilkan koordinat x, y, dan potongan gambar sesuai ukuran window_size
            yield x, y, image.crop((x, y, x + window_size[0], y + window_size[1]))

# Memuat model YOLOv8 yang sudah dilatih sebelumnya
model = YOLO('D:/KAMPUS/Model/.Model/contentv70/runs/detect/train/weights/best.pt')
# Menentukan path gambar yang akan diproses
image_path = 'D:/KAMPUS/Model/.Cliping/HutanPinus.png'
# Menentukan path file CHM yang akan digunakan
chm_path = 'D:/KAMPUS/Model/.CHM/CHM1.tif'
# Menentukan folder output untuk menyimpan hasil
output_folder = "D:/KAMPUS/Model/Export/ExportFilter"

# Membuka file gambar menggunakan rasterio
with rasterio.open(image_path) as src:
    # Membuka gambar menggunakan PIL
    image = Image.open(image_path)
    # Mendapatkan transformasi koordinat dari gambar
    transform = src.transform
    # Mendapatkan sistem referensi koordinat (CRS) dari gambar
    crs = src.crs

# Menentukan ukuran window tanpa padding
window_size = (2048, 2048)  # Ukuran window tanpa padding

# Menentukan ukuran langkah (step) untuk overlap antar window
step_size = 2048  # Overlap antar window

detected_points = []  # Inisialisasi list kosong untuk menyimpan titik-titik yang terdeteksi

with rasterio.open(chm_path) as chm_src:  # Membuka file CHM menggunakan rasterio
    chm_array = chm_src.read(1)  # Membaca data CHM sebagai array 2D
    height, width = chm_array.shape  # Mendapatkan tinggi dan lebar dari array CHM

    # Loop melalui setiap window yang dihasilkan oleh fungsi sliding_window
    for x, y, window in sliding_window(image, step_size, window_size):
        # Melakukan prediksi menggunakan model YOLO pada window dengan confidence threshold 0.10 dan IOU threshold 0.1
        results = model(window, conf=0.2, iou=0.1)
        # Mengubah hasil prediksi menjadi list jika hasilnya adalah generator
        results_list = list(results) if isinstance(results, types.GeneratorType) else results
        # Mendapatkan data bounding box dari hasil prediksi jika atribut 'data' ada
        boxes_data = results_list[0].boxes.data if hasattr(results_list[0].boxes, 'data') else []
        # Mengubah data bounding box menjadi list jika data tidak kosong
        boxes_list = boxes_data[:, :4].tolist() if boxes_data.nelement() else []

        # Loop melalui setiap bounding box dalam boxes_list
        for box in boxes_list:
            # Mendapatkan koordinat x1, y1, x2, y2 dari bounding box
            x1, y1, x2, y2 = box
            # Menghitung koordinat pusat bounding box dalam window
            center_x = (x1 + x2) / 2 + x
            center_y = (y1 + y2) / 2 + y
            # Mengubah koordinat pusat bounding box menjadi titik dalam sistem koordinat gambar
            point = Point(transform * (center_x, center_y))
            # Mendapatkan indeks baris dan kolom dari titik dalam array CHM
            row, col = chm_src.index(point.x, point.y)
            # Memeriksa apakah titik berada dalam batas array CHM dan nilai CHM di titik tersebut >= 1 meter
            if 0 <= row < height and 0 <= col < width and chm_array[row, col] >= 1:

                    # Menambahkan titik yang terdeteksi ke dalam list detected_points
                    detected_points.append(point)

def cluster_points(gdf, eps, min_samples):
    # Mengubah geometris dari GeoDataFrame menjadi array numpy dari koordinat x dan y
    coords = np.array(list(gdf.geometry.apply(lambda geom: (geom.x, geom.y))))
    # Melakukan clustering menggunakan algoritma DBSCAN dengan parameter eps dan min_samples
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    # Mendapatkan label hasil clustering
    labels = db.labels_

    # Membuat list dari titik-titik yang dikelompokkan berdasarkan label hasil clustering
    # Menggunakan koordinat rata-rata dari setiap kelompok (k) yang bukan noise (k != -1)
    clustered_points = [Point(coords[labels == k].mean(axis=0)) for k in set(labels) if k != -1]
    
    # Mengembalikan GeoDataFrame yang berisi titik-titik yang dikelompokkan dengan sistem koordinat yang sama dengan gdf
    return gpd.GeoDataFrame(geometry=clustered_points, crs=gdf.crs)

# Membuat GeoDataFrame dari list detected_points dengan sistem koordinat crs
gdf = gpd.GeoDataFrame(geometry=detected_points, crs=crs)
# Melakukan clustering pada GeoDataFrame dengan parameter eps dan min_samples
final_gdf = cluster_points(gdf, eps=2.46, min_samples=1)
# Menentukan path output untuk menyimpan file shapefile hasil clustering
final_output_path = os.path.join(output_folder, "Point_shapefile.shp")
# Menyimpan GeoDataFrame hasil clustering ke dalam file shapefile
final_gdf.to_file(final_output_path)

print("Proses prediksi dan clustering selesai!")


0: 640x640 (no detections), 300.0ms
Speed: 8.0ms preprocess, 300.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 297.0ms
Speed: 6.0ms preprocess, 297.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 189 Pines-Trees, 287.0ms
Speed: 5.0ms preprocess, 287.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 165 Pines-Trees, 277.0ms
Speed: 6.0ms preprocess, 277.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 33 Pines-Trees, 309.0ms
Speed: 5.0ms preprocess, 309.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 283.0ms
Speed: 6.0ms preprocess, 283.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 278.0ms
Speed: 8.0ms preprocess, 278.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 283.0ms
Speed: 6.0ms prepro

Proses prediksi dan clustering selesai!


Deteksi dan filtering chm


In [18]:
from ultralytics import YOLO
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point
import os
import rasterio
import numpy as np
from sklearn.cluster import DBSCAN
import types

Image.MAX_IMAGE_PIXELS = None  # Menghilangkan batasan piksel

# Fungsi untuk memotong gambar menjadi beberapa window tanpa padding
def sliding_window(image, step_size, window_size):
    # Loop melalui koordinat y dengan langkah sebesar step_size
    for y in range(0, image.height, step_size):
        # Loop melalui koordinat x dengan langkah sebesar step_size
        for x in range(0, image.width, step_size):
            # Menghasilkan koordinat x, y, dan potongan gambar sesuai ukuran window_size
            yield x, y, image.crop((x, y, x + window_size[0], y + window_size[1]))

# Memuat model YOLOv8 yang sudah dilatih sebelumnya
model = YOLO('D:/KAMPUS/Model/.Model/contentv70/runs/detect/train/weights/best.pt')
# Menentukan path gambar yang akan diproses
image_path = 'D:/KAMPUS/Model/.Cliping/HutanPinus.png'
# Menentukan path file CHM yang akan digunakan
chm_path = 'D:/KAMPUS/Model/.CHM/CHM1.tif'
# Menentukan folder output untuk menyimpan hasil
output_folder = "D:/KAMPUS/Model/Export/ExportTanpaFilter"

# Membuka file gambar menggunakan rasterio
with rasterio.open(image_path) as src:
    # Membuka gambar menggunakan PIL
    image = Image.open(image_path)
    # Mendapatkan transformasi koordinat dari gambar
    transform = src.transform
    # Mendapatkan sistem referensi koordinat (CRS) dari gambar
    crs = src.crs

# Menentukan ukuran window tanpa padding
window_size = (2048, 2048)  # Ukuran window tanpa padding

# Menentukan ukuran langkah (step) untuk overlap antar window
step_size = 2048  # Overlap antar window

detected_points = []  # Inisialisasi list kosong untuk menyimpan titik-titik yang terdeteksi

with rasterio.open(chm_path) as chm_src:  # Membuka file CHM menggunakan rasterio
    chm_array = chm_src.read(1)  # Membaca data CHM sebagai array 2D
    height, width = chm_array.shape  # Mendapatkan tinggi dan lebar dari array CHM

    # Loop melalui setiap window yang dihasilkan oleh fungsi sliding_window
    for x, y, window in sliding_window(image, step_size, window_size):
        # Melakukan prediksi menggunakan model YOLO pada window dengan confidence threshold 0.10 dan IOU threshold 0.1
        results = model(window, conf=0.2, iou=0.1)
        # Mengubah hasil prediksi menjadi list jika hasilnya adalah generator
        results_list = list(results) if isinstance(results, types.GeneratorType) else results
        # Mendapatkan data bounding box dari hasil prediksi jika atribut 'data' ada
        boxes_data = results_list[0].boxes.data if hasattr(results_list[0].boxes, 'data') else []
        # Mengubah data bounding box menjadi list jika data tidak kosong
        boxes_list = boxes_data[:, :4].tolist() if boxes_data.nelement() else []

        # Loop melalui setiap bounding box dalam boxes_list
        for box in boxes_list:
            # Mendapatkan koordinat x1, y1, x2, y2 dari bounding box
            x1, y1, x2, y2 = box
            # Menghitung koordinat pusat bounding box dalam window
            center_x = (x1 + x2) / 2 + x
            center_y = (y1 + y2) / 2 + y
            # Mengubah koordinat pusat bounding box menjadi titik dalam sistem koordinat gambar
            point = Point(transform * (center_x, center_y))
            # Mendapatkan indeks baris dan kolom dari titik dalam array CHM
            row, col = chm_src.index(point.x, point.y)
            # Memeriksa apakah titik berada dalam batas array CHM dan nilai CHM di titik tersebut >= 1 meter
            if 0 <= row < height and 0 <= col < width and chm_array[row, col] >= 0:
                    # Menambahkan titik yang terdeteksi ke dalam list detected_points
                    detected_points.append(point)

def cluster_points(gdf, eps, min_samples):
    # Mengubah geometris dari GeoDataFrame menjadi array numpy dari koordinat x dan y
    coords = np.array(list(gdf.geometry.apply(lambda geom: (geom.x, geom.y))))
    # Melakukan clustering menggunakan algoritma DBSCAN dengan parameter eps dan min_samples
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    # Mendapatkan label hasil clustering
    labels = db.labels_

    # Membuat list dari titik-titik yang dikelompokkan berdasarkan label hasil clustering
    # Menggunakan koordinat rata-rata dari setiap kelompok (k) yang bukan noise (k != -1)
    clustered_points = [Point(coords[labels == k].mean(axis=0)) for k in set(labels) if k != -1]
    
    # Mengembalikan GeoDataFrame yang berisi titik-titik yang dikelompokkan dengan sistem koordinat yang sama dengan gdf
    return gpd.GeoDataFrame(geometry=clustered_points, crs=gdf.crs)

# Membuat GeoDataFrame dari list detected_points dengan sistem koordinat crs
gdf = gpd.GeoDataFrame(geometry=detected_points, crs=crs)
# Melakukan clustering pada GeoDataFrame dengan parameter eps dan min_samples
final_gdf = cluster_points(gdf, eps=2.46, min_samples=1)
# Menentukan path output untuk menyimpan file shapefile hasil clustering
final_output_path = os.path.join(output_folder, "Point_shapefile.shp")
# Menyimpan GeoDataFrame hasil clustering ke dalam file shapefile
final_gdf.to_file(final_output_path)

print("Proses prediksi dan clustering selesai!")


0: 640x640 (no detections), 340.0ms
Speed: 7.0ms preprocess, 340.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 352.0ms
Speed: 8.0ms preprocess, 352.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 189 Pines-Trees, 330.0ms
Speed: 6.0ms preprocess, 330.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 165 Pines-Trees, 327.0ms
Speed: 6.0ms preprocess, 327.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 33 Pines-Trees, 341.0ms
Speed: 8.0ms preprocess, 341.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 316.0ms
Speed: 5.0ms preprocess, 316.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 300.0ms
Speed: 9.0ms preprocess, 300.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 303.0ms
Speed: 5.0ms prepro

Proses prediksi dan clustering selesai!
